# What patterns can we find in the data that are correlated with increases and decreases in the number of parking tickets issued by newyork city parking authority ? 

## importing necessary library

In [4]:
import dask.dataframe as dd # for lazy evaluation 
from dask.diagnostics import ProgressBar #to see progress of the process
import pandas as pd #data processing
import numpy as np # linear algebra
import seaborn as sns # for plotting the data

In [5]:
 #read data through dask 
df1 = dd.read_csv("./Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_.csv")
df2 = dd.read_csv("./Parking_Violations_Issued_-_Fiscal_Year_2015.csv")
df3 = dd.read_csv("./Parking_Violations_Issued_-_Fiscal_Year_2016.csv")
df4 = dd.read_csv("./Parking_Violations_Issued_-_Fiscal_Year_2017.csv")

In [14]:
df1.dtypes

Summons Number                         int64
Plate ID                              object
Registration State                    object
Plate Type                            object
Issue Date                            object
Violation Code                         int64
Vehicle Body Type                     object
Vehicle Make                          object
Issuing Agency                        object
Street Code1                           int64
Street Code2                           int64
Street Code3                           int64
Vehicle Expiration Date                int64
Violation Location                   float64
Violation Precinct                     int64
Issuer Precinct                        int64
Issuer Code                            int64
Issuer Command                        object
Issuer Squad                           int64
Violation Time                        object
Time First Observed                   object
Violation County                      object
Violation 

In [37]:
missing_values3 = df3.isnull().sum() # finding nan values in dataset columns

In [38]:
missing_values3

Dask Series Structure:
npartitions=1
BBL               int64
Violation Time      ...
dtype: int64
Dask Name: dataframe-sum-agg, 130 tasks

In [39]:
missing_count3 = ((missing_values3/df3.index.size)*100) 

In [40]:
missing_count3

Dask Series Structure:
npartitions=1
BBL               float64
Violation Time        ...
dtype: float64
Dask Name: mul, 219 tasks

In [41]:
with ProgressBar():
    missing_count_val3 = missing_count3.compute() #doing eager evaluation using compute function    

[                                        ] | 0% Completed | 27.7s


ValueError: Mismatched dtypes found in `pd.read_csv`/`pd.read_table`.

+-----------------------+--------+----------+
| Column                | Found  | Expected |
+-----------------------+--------+----------+
| Issuer Squad          | object | int64    |
| Violation Description | object | float64  |
| Violation Legal Code  | object | float64  |
| Violation Post Code   | object | float64  |
+-----------------------+--------+----------+

The following columns also raised exceptions on conversion:

- Issuer Squad
  ValueError("invalid literal for int() with base 10: 'K'")
- Violation Description
  ValueError("could not convert string to float: '38-Failure to Display Muni Rec'")
- Violation Legal Code
  ValueError("could not convert string to float: 'T'")
- Violation Post Code
  ValueError("could not convert string to float: '49 7'")

Usually this is due to dask's dtype inference failing, and
*may* be fixed by specifying dtypes manually by adding:

dtype={'Issuer Squad': 'object',
       '

C:\Users\HP\anaconda3\lib\multiprocessing\pool.py:125: DtypeWarning: Columns (17,18,20,21,22,23,29,30,31,32,36,38,39) have mixed types.Specify dtype option on import or set low_memory=False.
  result = (True, func(*args, **kwds))
C:\Users\HP\anaconda3\lib\multiprocessing\pool.py:125: DtypeWarning: Columns (17,18,20,21,22,23,30,31,32,36,38) have mixed types.Specify dtype option on import or set low_memory=False.
  result = (True, func(*args, **kwds))
C:\Users\HP\anaconda3\lib\multiprocessing\pool.py:125: DtypeWarning: Columns (18,38) have mixed types.Specify dtype option on import or set low_memory=False.
  result = (True, func(*args, **kwds))


In [ ]:
missing_count_val1

In [ ]:
columns_to_drop = missing_count_val[missing_count_val > 60].index #dropping the coulmns which have more than 60% nan values

In [ ]:
columns_to_drop # columns which are dropped

In [ ]:
with ProgressBar():
        df_dropped = df.drop(columns_to_drop,axis=1).persist() #save the data in memory by applying persist() function

# What patterns can we find in the data that are correlated with increases and decreases in the number of parking tickets issued by newyork city parking authority ? 

In [ ]:
fy14 = dd.read_csv("./Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_.csv")
fy15 = dd.read_csv("./Parking_Violations_Issued_-_Fiscal_Year_2015.csv")
fy16 = dd.read_csv("./Parking_Violations_Issued_-_Fiscal_Year_2016.csv")
fy17 = dd.read_csv("./Parking_Violations_Issued_-_Fiscal_Year_2017.csv")

In [ ]:
from functools import reduce

In [ ]:
columns = [set(fy14.columns),set(fy15.columns),set(fy16.columns),set(fy17.columns)]
common_columns = list(reduce(lambda x,y: x.intersection(y),columns))  # making the list of common_columns

In [ ]:
fy15=fy15[common_columns]

In [ ]:
fy15.dtypes #fy15 columns datatypes

In [ ]:
fy15=fy15[common_columns]

In [ ]:
fy16=fy16[common_columns]

In [ ]:
fy17=fy17[common_columns]

In [ ]:
dtype_tuples = [(x,np.str) for x in common_columns] #convert common_columns datatye into string

dtypes = dict(dtype_tuples)

In [ ]:
fy14 = dd.read_csv("./Parking_Violations_Issued_-_Fiscal_Year_2014__August_2013___June_2014_.csv",dtype=dtypes)
fy15 = dd.read_csv("./Parking_Violations_Issued_-_Fiscal_Year_2015.csv",dtype=dtypes)
fy16 = dd.read_csv("./Parking_Violations_Issued_-_Fiscal_Year_2016.csv",dtype=dtypes)
fy17 = dd.read_csv("./Parking_Violations_Issued_-_Fiscal_Year_2017.csv",dtype=dtypes)

In [ ]:
complete_data = dd.read_csv("./*.csv",dtype=dtypes,usecols=common_columns) #read all common_columns data of fy14,fy15,fy16,fy17

In [ ]:
complete_data.dtypes #datatype of common_columns after converting into string

In [ ]:
int_datatype=['Summons Number','Violation Code','Street Code1', 'Street Code2', 'Street Code3',
       'Vehicle Expiration Date', 'Violation Precinct','Issuer Precinct', 'Issuer Code','Date First Observed',
        'Law Section','Vehicle Year','Feet From Curb']   # make a list of int64 dtype columns

In [ ]:
float_datatype=['Violation Location','Unregistered Vehicle?','No Standing or Stopping Violation', 'Hydrant Violation',
       'Double Parking Violation']   # make a list of float64 datatype

In [ ]:
complete_data[int_datatype]=complete_data[int_datatype].astype('int64') #converting data into int dtypes

In [ ]:
complete_data[float_datatype]=complete_data[float_datatype].astype('float64') # converting data into float dtypes

In [ ]:
complete_data.dtypes

In [ ]:
complete_data.head()

In [ ]:
new_data=complete_data.corr( method='pearson') #excluding nan values from the datasets

In [ ]:
new_data = dd.from_pandas(df, npartitions=3) 


In [ ]:
nyc_data_raw = dd.read_csv("./*.csv",dtype=dtypes, usecols=dtypes.keys())